## Comprobar HW y SW instalado








In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   2
  On-line CPU(s) list:    0,1
Vendor ID:                GenuineIntel
  Model name:             Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:           6
    Model:                85
    Thread(s) per core:   2
    Core(s) per socket:   1
    Socket(s):            1
    Stepping:             3
    BogoMIPS:             4000.42
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 cl
                          flush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc re
                          p_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3
                           fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand
                           hypervisor lahf_lm abm 3dnowprefetch i

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [3]:
!nvidia-smi

Tue Oct  8 09:38:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Compilar ejecutar un program (GoogleColab con GPUs)



In [5]:
! mkdir cuda

In [6]:
%%writefile cuda/holamundo.cu

#include <stdio.h>
#include <cuda.h>
#include <cuda_runtime.h>

__global__ void mykernel(void) {

}

int main(void) {
		mykernel<<<1,1>>>();
		printf("Hello world\n");
		return 0;
}

Writing cuda/holamundo.cu


In [7]:
!ls


cuda  sample_data


In [8]:
!nvcc cuda/holamundo.cu -o cuda/holamundo
!ls -la cuda

total 960
drwxr-xr-x 2 root root   4096 Oct  8 09:51 .
drwxr-xr-x 1 root root   4096 Oct  8 09:45 ..
-rwxr-xr-x 1 root root 967080 Oct  8 09:51 holamundo
-rw-r--r-- 1 root root    183 Oct  8 09:45 holamundo.cu


In [9]:
!./cuda/holamundo

Hello world


In [10]:
%%writefile cuda/suma1d.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

int main(void) {
    int a, b, c;				    // host copies of a, b, c
	  int *d_a, *d_b, *d_c;		// device copies of a, b, c
	  int size = sizeof(int);

	  // Allocate space for device copies of a, b, c
	  cudaMalloc((void **)&d_a, size);
	  cudaMalloc((void **)&d_b, size);
	  cudaMalloc((void **)&d_c, size);

	  // Setup input values
	  a = 2;
	  b = 7;

	  // Copy inputs to device
	  cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	  cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	  // Launch add() kernel on GPU
	  add<<<1,1>>>(d_a, d_b, d_c);

	  // Copy result back to host
	  cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	  // Show result
	  printf("%d + %d = %d", a, b, c);

	  // Cleanup
	  cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	  return 0;
}

Writing cuda/suma1d.cu


In [11]:
!nvcc cuda/suma1d.cu -o cuda/suma1d
!ls -la cuda

total 1912
drwxr-xr-x 2 root root   4096 Oct  8 09:54 .
drwxr-xr-x 1 root root   4096 Oct  8 09:45 ..
-rwxr-xr-x 1 root root 967080 Oct  8 09:51 holamundo
-rw-r--r-- 1 root root    183 Oct  8 09:45 holamundo.cu
-rwxr-xr-x 1 root root 967072 Oct  8 09:54 suma1d
-rw-r--r-- 1 root root    906 Oct  8 09:53 suma1d.cu


In [12]:
!./cuda/suma1d

2 + 7 = 9

Comprobar perfil de ejecución

In [30]:
!nvprof cuda/suma1d

==8722== NVPROF is profiling process 8722, command: cuda/suma1d
2 + 7 = 9==8722== Profiling application: cuda/suma1d
==8722== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   45.81%  3.3280us         1  3.3280us  3.3280us  3.3280us  add(int*, int*, int*)
                   29.07%  2.1120us         1  2.1120us  2.1120us  2.1120us  [CUDA memcpy DtoH]
                   25.11%  1.8240us         2     912ns     672ns  1.1520us  [CUDA memcpy HtoD]
      API calls:   99.49%  93.397ms         3  31.132ms  4.2520us  93.387ms  cudaMalloc
                    0.19%  176.61us         1  176.61us  176.61us  176.61us  cudaLaunchKernel
                    0.14%  136.10us       114  1.1930us     132ns  53.862us  cuDeviceGetAttribute
                    0.10%  97.772us         3  32.590us  3.5840us  86.293us  cudaFree
                    0.05%  48.434us         3  16.144us  7.1390us  21.053us  cudaMemcpy
                    0.01%  10

### Compilar y ejecutar un programa (code in multiple source files)

In [14]:
%%writefile cuda/kernel.cu
#include <cuda.h>

__global__ void add(int *a, int *b, int *c) {
    *c = *a + *b;
}

Writing cuda/kernel.cu


In [16]:
%%writefile cuda/main.cu
#include <cuda.h>
#include <cuda_runtime.h>
#include <stdio.h>

__global__ void add(int *a, int *b, int *c);

int main(void) {
	int a, b, c;				    // host copies of a, b, c
	int *d_a, *d_b, *d_c;		// device copies of a, b, c
	int size = sizeof(int);

	// Allocate space for device copies of a, b, c
	cudaMalloc((void **)&d_a, size);
	cudaMalloc((void **)&d_b, size);
	cudaMalloc((void **)&d_c, size);

	// Setup input values
	a = 2;
	b = 7;

	// Copy inputs to device
	cudaMemcpy(d_a, &a, size, cudaMemcpyHostToDevice);
	cudaMemcpy(d_b, &b, size, cudaMemcpyHostToDevice);

	// Launch add() kernel on GPU
	add<<<1,1>>>(d_a, d_b, d_c);

	// Copy result back to host
	cudaMemcpy(&c, d_c, size, cudaMemcpyDeviceToHost);

	// Show result
	printf("%d + %d = %d", a, b, c);

	// Cleanup
	cudaFree(d_a); cudaFree(d_b); cudaFree(d_c);
	return 0;
}

Overwriting cuda/main.cu


In [17]:
!nvcc cuda/kernel.cu cuda/main.cu -o cuda/program
!ls -la cuda

total 2868
drwxr-xr-x 2 root root   4096 Oct  8 09:56 .
drwxr-xr-x 1 root root   4096 Oct  8 09:45 ..
-rwxr-xr-x 1 root root 967080 Oct  8 09:51 holamundo
-rw-r--r-- 1 root root    183 Oct  8 09:45 holamundo.cu
-rw-r--r-- 1 root root     85 Oct  8 09:55 kernel.cu
-rw-r--r-- 1 root root    840 Oct  8 09:56 main.cu
-rwxr-xr-x 1 root root 967496 Oct  8 09:56 program
-rwxr-xr-x 1 root root 967072 Oct  8 09:54 suma1d
-rw-r--r-- 1 root root    906 Oct  8 09:53 suma1d.cu


In [18]:
!./cuda/program

2 + 7 = 9

## Mount your Google Drive

In [19]:
# Mount Google Drive
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
! ls -la gdrive/MyDrive/Colab\ Notebooks

In [ ]:
# Copy source code to your folder
!cp /content/gdrive/MyDrive/XXX/* cuda/
# Create soft links
!ln -s /content/gdrive/MyDrive/XXXX cuda/XXXX
# List folder
!ls -la cuda/

## Compilar ejemplos de CUDA

In [80]:
cd /content/cuda

/content/cuda


In [81]:
!git clone https://github.com/NVIDIA/cuda-samples.git

Cloning into 'cuda-samples'...
remote: Enumerating objects: 19507, done.
remote: Counting objects: 100% (10080/10080), done.
remote: Compressing objects: 100% (569/569), done.
remote: Total 19507 (delta 9605), reused 9714 (delta 9511), pack-reused 9427 (from 1)
Receiving objects: 100% (19507/19507), 133.56 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (17060/17060), done.
Updating files: 100% (4026/4026), done.


In [92]:
ls /content/cuda/cuda-samples/Samples/*

/content/cuda/cuda-samples/Samples/0_Introduction:
asyncAPI/                simpleAtomicIntrinsics_nvrtc/  simpleSeparateCompilation/
c++11_cuda/              simpleAttributes/              simpleStreams/
clock/                   simpleAWBarrier/               simpleSurfaceWrite/
clock_nvrtc/             simpleCallback/                simpleTemplates/
concurrentKernels/       simpleCooperativeGroups/       simpleTemplates_nvrtc/
cppIntegration/          simpleCubemapTexture/          simpleTexture/
cppOverload/             simpleCUDA2GL/                 simpleTexture3D/
cudaOpenMP/              simpleDrvRuntime/              simpleTextureDrv/
fp16ScalarProduct/       simpleHyperQ/                  simpleVoteIntrinsics/
matrixMul/               simpleIPC/                     simpleVoteIntrinsics_nvrtc/
matrixMulDrv/            simpleLayeredTexture/          simpleZeroCopy/
matrixMulDynlinkJIT/     simpleMPI/                     systemWideAtomics/
matrixMul_nvrtc/         simpleMultiCopy

In [83]:
cd /content/cuda/cuda-samples/Samples/1_Utilities/deviceQuery

/content/cuda/cuda-samples/Samples/1_Utilities/deviceQuery


In [84]:
!ls -la

total 88
drwxr-xr-x 3 root root  4096 Oct  8 10:38 .
drwxr-xr-x 6 root root  4096 Oct  8 10:38 ..
-rw-r--r-- 1 root root 13921 Oct  8 10:38 deviceQuery.cpp
-rw-r--r-- 1 root root   851 Oct  8 10:38 deviceQuery_vs2017.sln
-rw-r--r-- 1 root root  5199 Oct  8 10:38 deviceQuery_vs2017.vcxproj
-rw-r--r-- 1 root root   851 Oct  8 10:38 deviceQuery_vs2019.sln
-rw-r--r-- 1 root root  4784 Oct  8 10:38 deviceQuery_vs2019.vcxproj
-rw-r--r-- 1 root root   851 Oct  8 10:38 deviceQuery_vs2022.sln
-rw-r--r-- 1 root root  4784 Oct  8 10:38 deviceQuery_vs2022.vcxproj
-rw-r--r-- 1 root root 12973 Oct  8 10:38 Makefile
-rw-r--r-- 1 root root  2154 Oct  8 10:38 NsightEclipse.xml
-rw-r--r-- 1 root root  3639 Oct  8 10:38 README.md
drwxr-xr-x 2 root root  4096 Oct  8 10:38 .vscode


In [85]:
!make

/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda/bin/nvcc -ccbin g++ -m64 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o deviceQuery deviceQuery.o 
mkdir -p

In [86]:
!./deviceQuery

./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          12.2 / 12.2
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15102 MBytes (15835660288 bytes)
  (040) Multiprocessors, (064) CUDA Cores/MP:    2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shar

In [88]:
cd /content/cuda/cuda-samples/Samples/1_Utilities/topologyQuery

/content/cuda/cuda-samples/Samples/1_Utilities/topologyQuery


In [89]:
!make

/usr/local/cuda/bin/nvcc -ccbin g++ -I../../../Common -m64 --threads 0 --std=c++11 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o topologyQuery.o -c topologyQuery.cu
/usr/local/cuda/bin/nvcc -ccbin g++ -m64 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_89,code=sm_89 -gencode arch=compute_90,code=sm_90 -gencode arch=compute_90,code=compute_90 -o topologyQuery topologyQuery.o 
m

In [90]:
!ls -la

total 1056
drwxr-xr-x 3 root root   4096 Oct  8 10:41 .
drwxr-xr-x 6 root root   4096 Oct  8 10:38 ..
-rw-r--r-- 1 root root  13540 Oct  8 10:38 Makefile
-rw-r--r-- 1 root root   1908 Oct  8 10:38 NsightEclipse.xml
-rw-r--r-- 1 root root   3314 Oct  8 10:38 README.md
-rwxr-xr-x 1 root root 979960 Oct  8 10:41 topologyQuery
-rw-r--r-- 1 root root   3411 Oct  8 10:38 topologyQuery.cu
-rw-r--r-- 1 root root  17568 Oct  8 10:41 topologyQuery.o
-rw-r--r-- 1 root root    855 Oct  8 10:38 topologyQuery_vs2017.sln
-rw-r--r-- 1 root root   5208 Oct  8 10:38 topologyQuery_vs2017.vcxproj
-rw-r--r-- 1 root root    855 Oct  8 10:38 topologyQuery_vs2019.sln
-rw-r--r-- 1 root root   4793 Oct  8 10:38 topologyQuery_vs2019.vcxproj
-rw-r--r-- 1 root root    855 Oct  8 10:38 topologyQuery_vs2022.sln
-rw-r--r-- 1 root root   4793 Oct  8 10:38 topologyQuery_vs2022.vcxproj
drwxr-xr-x 2 root root   4096 Oct  8 10:38 .vscode


In [91]:
!./topologyQuery

GPU0 <-> CPU:
  * Atomic Supported: no
